In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt
# import tensorflow as tf
import os

In [ ]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

In [ ]:

input_video_path = '/Users/suryanshpatel/Projects/Pose_detection/videos/pullupsboth.mp4'
output_folder = '/Users/suryanshpatel/Projects/Pose_detection/Frames/annotated_frames/af7'
coordinates_folder = '/Users/suryanshpatel/Projects/Pose_detection/Frames/plots/co7'
output_video_path = '/Users/suryanshpatel/Projects/Pose_detection/videos/pullUps_7.mp4'

main code in next 2 blocks

In [ ]:
def is_key_frame(prev_frame, current_frame, threshold=30):
    diff = cv2.absdiff(prev_frame, current_frame)
    non_zero_count = np.count_nonzero(diff)
    return non_zero_count > threshold

In [ ]:
# Open video file
key_frames = []
prev_frame = None
cap = cv2.VideoCapture(0)

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Initialize VideoWriter
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for mp4 files
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# List to store all joint points
all_joint_points = []

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    # Convert the BGR image to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    if prev_frame is None or is_key_frame(prev_frame, gray_frame):
        key_frames.append(frame)
        prev_frame = gray_frame


    # Process the image and detect the pose
    results = pose.process(rgb_frame)

    # Capture joint points
    joint_points = []
    if results.pose_landmarks:
        for landmark in results.pose_landmarks.landmark:
            joint_points.append((landmark.x, landmark.y))

    all_joint_points.append(joint_points)


    # Convert the image back to BGR for rendering
    image = cv2.cvtColor(rgb_frame, cv2.COLOR_RGB2BGR)

    # Draw the pose annotation on the image
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    # Write the frame to the output video
    out.write(image)

    # Optionally display the image (comment out if not needed)
    cv2.imshow('Pose Detection', image)
    if cv2.waitKey(5) & 0xFF == 27:  # Press 'Esc' to exit
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()  # Uncomment if you are displaying the video

# Convert to NumPy array
all_joint_points = np.array(all_joint_points)

print(f'Output video saved as {output_video_path}')

In [ ]:
arr = np.array(key_frames)
arr.shape

195: The number of frames in the video.
480: The height of each frame in pixels.
270: The width of each frame in pixels.
2: The number of color channels (typically RGB, where 3 channels represent Red, Green, and Blue).

In [ ]:
arr2 = np.array(all_joint_points)
print(arr2.shape)
print(arr2[0].shape)

195: Number of frames in your video.
33: Number of joints detected in each frame.
2: Each joint has 2 coordinates, typically representing (x, y).

saves video skeleton
saves frames - frame of every picture
saves plots - plots for given frames


In [ ]:
# Function to plot and save joint coordinates
def plot_coordinates(coordinates, frame_number, frame_width, frame_height, save_folder):
    plt.figure()
    
    # Scale coordinates to frame dimensions
    scaled_coordinates = [(x * frame_width, y * frame_height) for x, y in coordinates]

    # Plot scaled coordinates
    xs, ys = zip(*scaled_coordinates)
    plt.plot(xs, ys, marker='o', linestyle='-', color='b')
    plt.title(f'Joint Coordinates - Frame {frame_number}')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.gca().invert_yaxis()  # Invert y-axis to match image coordinates
    plt.xlim(0, frame_width)
    plt.ylim(frame_height, 0)  # Invert y-axis limits
    plt.savefig(os.path.join(save_folder, f'coordinates_frame_{frame_number:03d}.png'))
    plt.close()


In [ ]:
# Create output folders if they don't exist
os.makedirs(output_folder, exist_ok=True)
os.makedirs(coordinates_folder, exist_ok=True)

# Initialize video capture
cap = cv2.VideoCapture(0)

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Initialize VideoWriter for annotated video

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Initialize variables for key frames
key_frames = []
prev_frame = None

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    # Convert the BGR image to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    if prev_frame is None or is_key_frame(prev_frame, gray_frame):
        key_frames.append(frame)
        prev_frame = gray_frame

    # Process the image and detect the pose
    results = pose.process(rgb_frame)

    # Create a blank image (white background)
    blank_image = np.ones((frame_height, frame_width, 3), dtype=np.uint8) * 255

    # Draw the pose annotation on the blank image
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(blank_image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Save annotated frame
        annotated_frame_path = os.path.join(output_folder, f'frame_{len(key_frames):03d}.png')
        cv2.imwrite(annotated_frame_path, cv2.cvtColor(blank_image, cv2.COLOR_RGB2BGR))

        # Plot and save coordinates
        if results.pose_landmarks:
            joint_coordinates = [(lm.x, lm.y) for lm in results.pose_landmarks.landmark]
            plot_coordinates(joint_coordinates, frame_number=len(key_frames), 
                             frame_width=frame_width, frame_height=frame_height, save_folder=coordinates_folder)

    # Write the frame to the output video
    out.write(blank_image)

# Release resources
cap.release()
out.release()
# cv2.destroyAllWindows()  # Uncomment if you are displaying the video

print(f'Output video saved as {output_video_path}')
print(f'Annotated frames saved in {output_folder}')
print(f'Coordinate plots saved in {coordinates_folder}')

automatic exercise detection attepmt

In [ ]:
all_joint_points.shape

In [ ]:
all_joint_points[0][28]